Alerta de ordens pendentes criacao de KPI:
ordens pendentes, quantidade de itens e infos de clientes para envio de alerta por email e telefone de quem tem o cadastro completo

In [0]:
%python
bronze_path = "abfss://uc-external-curso-databricks-vf@sacursodatabricksvf.dfs.core.windows.net/bikestore/bronze/"
silver_path = "abfss://uc-external-curso-databricks-vf@sacursodatabricksvf.dfs.core.windows.net/bikestore/silver/"
gold_path = "abfss://uc-external-curso-databricks-vf@sacursodatabricksvf.dfs.core.windows.net/bikestore/gold/"
resource_path = "abfss://uc-external-curso-databricks-vf@sacursodatabricksvf.dfs.core.windows.net/bikestore/resource/origem"
resource_path_volume = "/Volumes/bikestore/logistica/bikestore_resource/origem/"

In [0]:
%python
silver_map= {
    "tmp_silver_custumer": f"{silver_path}/customer/",
    "tmp_silver_orders": f"{silver_path}/orders/",
    # "tmp_silver_product": f"{silver_path}/product/",
}
for view_name, path in silver_map.items():
    (spark.read.format("delta")
     .load(path)
     .createOrReplaceTempView(view_name))

In [0]:
SELECT * FROM tmp_silver_orders LIMIT 10

In [0]:
with pending as (

  SELECT
customer_id,
order_date,
sum(quantity) as quantity,
store_name
-- lower(status) as status
FROM tmp_silver_orders
WHERE 1=1
AND lower(status) = "pending"
GROUP BY customer_id, order_date, store_name
--, status

)

SELECT
  p.*, 
  c.first_name as first_name_customer,
  c.last_name,
  c.email,
  c.phone
FROM pending p
LEFT JOIN tmp_silver_custumer c
  ON p.customer_id = c.customer_id
WHERE 1=1
AND c.email IS NOT NULL
AND c.phone IS NOT NULL


In [0]:
%python
df_gold_orders_peding = spark.sql("""
with pending as (

  SELECT
customer_id,
order_date,
sum(quantity) as quantity,
store_name
-- lower(status) as status
FROM tmp_silver_orders
WHERE 1=1
AND lower(status) = "pending"
GROUP BY customer_id, order_date, store_name
--, status

)

SELECT
  p.*, 
  c.first_name as first_name_customer,
  c.last_name,
  c.email,
  c.phone
FROM pending p
LEFT JOIN tmp_silver_custumer c
  ON p.customer_id = c.customer_id
WHERE 1=1
AND c.email IS NOT NULL
AND c.phone IS NOT NULL                        
                                   """)

# salvando em delta na Silver
df_gold_orders_peding.write.mode("overwrite").format("delta").save(f"{gold_path}/orders_pending")

In [0]:
CREATE TABLE IF NOT EXISTS bikestore.logistica.gold_orders_pending
LOCATION "abfss://uc-external-curso-databricks-vf@sacursodatabricksvf.dfs.core.windows.net/bikestore/gold/orders_pending"

In [0]:
SELECT * FROM bikestore.logistica.gold_orders_pending